In [21]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"


In [22]:
import pandas as pd

DATA_PATH = "../Data"

# Load the data from the TSV files
arguments_data = pd.read_csv(f'{DATA_PATH}/arguments-training.tsv', delimiter='\t')
labels_data = pd.read_csv(f'{DATA_PATH}/labels-training.tsv', delimiter='\t')

# Display the first few rows of each dataframe to understand their structure
# print dimensions of the data
print(f'Arguments data shape: {arguments_data.shape}')
arguments_data.head()

Arguments data shape: (5393, 4)


,Argument ID,Conclusion,Stance,Premise
0,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...
1,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...
2,A01006,We should end the use of economic sanctions,against,sometimes economic sanctions are the only thin...
3,A01007,We should abolish capital punishment,against,capital punishment is sometimes the only optio...
4,A01008,We should ban factory farming,against,factory farming allows for the production of c...


In [23]:
# print dimensions of the data
print(f'Labels data shape: {labels_data.shape}')
labels_data.head()

Labels data shape: (5393, 21)


,Argument ID,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,...,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,A01002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A01005,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,A01006,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A01007,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,A01008,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0


In [24]:
# Step 1: Merge the datasets on "Argument ID"
full_data = pd.merge(arguments_data, labels_data, on="Argument ID")

# Step 2: Prepare the Input
# Convert labels to a string format where each label is prefixed by its name for clarity
labels_columns = labels_data.columns[1:]  # skip 'Argument ID'
full_data['Input'] = (full_data['Stance'] + " " + full_data['Premise'] + " " +
                      full_data[labels_columns].apply(
                          lambda row: ' '.join([f"{col}:{val}" for col, val in row.items()]), axis=1))

# Output is already in the "Conclusion" column, so no changes are needed there

# Step 3: Display the final format
final_data_preview = full_data[['Input', 'Conclusion']].head()

final_data_preview
# convert to pandas dataframe
final_data = pd.DataFrame(full_data[['Input', 'Conclusion']])

In [25]:
# print dimensions of the data
print(f'Final data shape: {final_data.shape}')
final_data.head()

Final data shape: (5393, 2)


,Input,Conclusion
0,in favor of we should ban human cloning as it ...,We should ban human cloning
1,in favor of fast food should be banned because...,We should ban fast food
2,against sometimes economic sanctions are the o...,We should end the use of economic sanctions
3,against capital punishment is sometimes the on...,We should abolish capital punishment
4,against factory farming allows for the product...,We should ban factory farming


In [26]:
# print input of first element of the data
print(f'Input of the first element: {final_data["Input"][0]}')

Input of the first element: in favor of we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same. Self-direction: thought:0 Self-direction: action:0 Stimulation:0 Hedonism:0 Achievement:0 Power: dominance:0 Power: resources:0 Face:0 Security: personal:0 Security: societal:1 Tradition:0 Conformity: rules:0 Conformity: interpersonal:0 Humility:0 Benevolence: caring:0 Benevolence: dependability:0 Universalism: concern:0 Universalism: nature:0 Universalism: tolerance:0 Universalism: objectivity:0


In [32]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch

# Ensure data types for proper processing
full_data['Input'] = full_data['Input'].astype(str)
full_data['Conclusion'] = full_data['Conclusion'].astype(str)

# Split the data into training and validation sets
train_data, val_data = train_test_split(full_data, test_size=0.1, random_state=42)

# Create a custom dataset class
class ConclusionDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        input_text = row['Input']
        target_text = row['Conclusion']

        # Tokenize the inputs and targets
        input_tokens = self.tokenizer.encode_plus(input_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        target_tokens = self.tokenizer.encode_plus(target_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')

        return {
            'input_ids': input_tokens['input_ids'].flatten(),
            'attention_mask': input_tokens['attention_mask'].flatten(),
            'labels': target_tokens['input_ids'].flatten()
        }
    
class EmphasizedConclusionDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]

        # print columns of the row
        # print(row.keys())

        # print the content of the row
        # print(row)

        STANCE = row['Stance']
        PREMISE = row['Premise']
        # INPUT_LABELS = row['Input Labels']

        # iterate over labels from columns 4 to 23 and create a string representation of the labels
        INPUT_LABELS = ''
        for i in range(4, 24):
            INPUT_LABELS += f'{row.keys()[i]}:{row[i]} '

        # print the input labels
            


        # Emphasizing the stance by repeating it
        input_text = f"[STANCE] {row['Stance']} [STANCE] {row['Premise']} Labels: {INPUT_LABELS} [STANCE] {row['Stance']} [STANCE]"
        target_text = row['Conclusion']

        # Tokenize the inputs and targets
        input_tokens = self.tokenizer.encode_plus(input_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        target_tokens = self.tokenizer.encode_plus(target_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')

        return {
            'input_ids': input_tokens['input_ids'].flatten(),
            'attention_mask': input_tokens['attention_mask'].flatten(),
            'labels': target_tokens['input_ids'].flatten()
        }




In [33]:
# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Prepare the datasets for DataLoader
train_dataset = ConclusionDataset(tokenizer, train_data)
val_dataset = ConclusionDataset(tokenizer, val_data)

# You would use this dataset instead of the basic ConclusionDataset for both training and validation
train_dataset = EmphasizedConclusionDataset(tokenizer, train_data)
val_dataset = EmphasizedConclusionDataset(tokenizer, val_data)


# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Example data from loader
next(iter(train_loader))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Argument ID                                                              A29002
Conclusion                              We should adopt gender-neutral language
Stance                                                                  against
Premise                       gender-neutral pronouns won't catch on,  it is...
Self-direction: thought                                                       0
Self-direction: action                                                        0
Stimulation                                                                   0
Hedonism                                                                      0
Achievement                                                                   0
Power: dominance                                                              0
Power: resources                                                              0
Face                                                                          0
Security: personal                      

KeyError: 'Input Labels'

In [ ]:
from transformers import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "mps")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
    model.train()
    for i, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:  # print every 10 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {loss.item()}")

print('Finished Training')


/Users/jyotir/miniforge3/envs/NLP_A3/lib/python3.11/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


[1, 1] loss: 20.75670623779297
[1, 11] loss: 3.8392670154571533
[1, 21] loss: 1.2759162187576294
[1, 31] loss: 1.1407668590545654
[1, 41] loss: 0.9949736595153809
[1, 51] loss: 0.7823083400726318
[1, 61] loss: 0.4907335042953491
[1, 71] loss: 0.374162882566452
[1, 81] loss: 0.2553357481956482
[1, 91] loss: 0.23939649760723114
[1, 101] loss: 0.21432092785835266
[1, 111] loss: 0.5390495657920837
[1, 121] loss: 0.18568634986877441
[1, 131] loss: 0.1825253963470459
[1, 141] loss: 0.19272756576538086
[1, 151] loss: 0.18500098586082458
[1, 161] loss: 0.15291279554367065
[1, 171] loss: 0.1490883231163025
[1, 181] loss: 0.16824324429035187
[1, 191] loss: 0.13727515935897827
[1, 201] loss: 0.1279458999633789
[1, 211] loss: 0.16480141878128052
[1, 221] loss: 0.13407132029533386
[1, 231] loss: 0.13001631200313568
[1, 241] loss: 0.10458501428365707
[1, 251] loss: 0.12684358656406403
[1, 261] loss: 0.11170413345098495
[1, 271] loss: 0.1137695163488388
[1, 281] loss: 0.12016907334327698
[1, 291] los

In [ ]:
# ! pip install sacrebleu

In [ ]:
# import sacrebleu

# def evaluate_bleu(model, data_loader, tokenizer, device):
#     model.eval()
#     refs, hyps = [], []
#     with torch.no_grad():
#         for batch in data_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)

#             # Generate output sequences
#             outputs = model.generate(input_ids, attention_mask=attention_mask)
#             for ref, hyp in zip(labels, outputs):
#                 ref_text = tokenizer.decode(ref, skip_special_tokens=True)
#                 hyp_text = tokenizer.decode(hyp, skip_special_tokens=True)
#                 refs.append([ref_text])
#                 hyps.append(hyp_text)

#     bleu_score = sacrebleu.corpus_bleu(hyps, refs)
#     return bleu_score.score

In [ ]:
import evaluate

from torch.nn.functional import softmax

def generate_text(input_text, model, tokenizer, device, max_length=512):
    model.eval()
    input_tokens = tokenizer.encode(input_text, return_tensors='pt').to(device)
    outputs = model.generate(input_tokens, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



def evaluate_bleu(model, data_loader, tokenizer, device):
    model.eval()
    refs, hyps = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Generate output sequences
            outputs = model.generate(input_ids, attention_mask=attention_mask)
            for ref, hyp in zip(labels, outputs):
                ref_text = tokenizer.decode(ref, skip_special_tokens=True)
                hyp_text = tokenizer.decode(hyp, skip_special_tokens=True)
                refs.append(ref_text)
                hyps.append(hyp_text)

    bleu_score = evaluate.load('bleu')
    results = bleu_score.compute(predictions=hyps, references=refs)

    return results


    # return bleu_score



In [29]:
# Assuming you have a 'device' and 'model' already set up and moved to the device
# Example usage:


#  = "in favor of we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same. Self-direction: thought:0 Self-direction: action:0 Stimulation:0 Hedonism:0 Achievement:0 Power: dominance:0 Power: resources:0 Face:0 Security: personal:0 Security: societal:1 Tradition:0 Conformity: rules:0 Conformity: interpersonal:0 Humility:0 Benevolence: caring:0 Benevolence: dependability:0 Universalism: concern:0 Universalism: nature:0 Universalism: tolerance:0 Universalism: objectivity:0"
# use sample input for testing from the dataset
input_inputdata = train_data['Input'].values
conclusion_inputdata = train_data['Conclusion'].values

for i in range(20):
    sample_input = input_inputdata[i]
    sample_conclusion = conclusion_inputdata[i]
    print("Input:", sample_input)
    print("Expected Conclusion:", sample_conclusion)
    print("Generated Conclusion:", generate_text(sample_input, model, tokenizer, device))
    print()


# print("Generated Conclusion:", generate_text(sample_input, model, tokenizer, device))



Input: in favor of school uniforms restrict students to make their own decisions and choices Self-direction: thought:1 Self-direction: action:1 Stimulation:1 Hedonism:0 Achievement:0 Power: dominance:0 Power: resources:0 Face:1 Security: personal:0 Security: societal:0 Tradition:0 Conformity: rules:0 Conformity: interpersonal:0 Humility:0 Benevolence: caring:0 Benevolence: dependability:0 Universalism: concern:1 Universalism: nature:0 Universalism: tolerance:0 Universalism: objectivity:0
Expected Conclusion: We should abandon the use of school uniform


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [20]:
# To evaluate on the validation set:
print("BLEU Score:", evaluate_bleu(model, val_loader, tokenizer, device))

/Users/jyotir/miniforge3/envs/NLP_A3/lib/python3.11/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


BLEU Score: {'bleu': 0.3335107183458902, 'precisions': [0.6476771787046457, 0.4761311747613117, 0.3097913322632424, 0.2520692249811889], 'brevity_penalty': 0.8466252571878353, 'length_ratio': 0.8572674418604651, 'translation_length': 2949, 'reference_length': 3440}


In [ ]:
BLEU Score: {'bleu': 0.3335107183458902, 'precisions': [0.6476771787046457, 0.4761311747613117, 0.3097913322632424, 0.2520692249811889], 'brevity_penalty': 0.8466252571878353, 'length_ratio': 0.8572674418604651, 'translation_length': 2949, 'reference_length': 3440}